In [1]:
%config IPCompleter.greedy=True

from os import path
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.6.0
False


In [2]:
import numpy as np
import torchvision
from PIL import Image

from matplotlib import pyplot as plt;
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from torch.utils.data import DataLoader

In /home/lukinio/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lukinio/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lukinio/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/lukinio/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/lukinio/anacond

In [3]:
from utils import *
from attacks import fgsm, bim, llc
from train_utils import train_model
from models.models import *

from nupic.torch.modules import (KWinners, SparseWeights, Flatten, SparseWeights2d, KWinners2d, 
                                 rezero_weights, update_boost_strength)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
transform = ToTensor()
ds = FashionMNIST('./data', train=True, target_transform=None, download=True, transform=transform)
ds_test = FashionMNIST('./data', train=False, target_transform=None, download=True, transform=transform)

In [6]:
test1000 = DataLoader(ds_test, 1000, shuffle=True)
for X, y in test1000:
    X, y = X.to(device), y.to(device)
    break

In [7]:
fcnet = FCNet()
sparse_fcnet = SparseFCNet()

fcnet.load_state_dict(torch.load("models/fcnet.pt", map_location=device))
sparse_fcnet.load_state_dict(torch.load("models/sparse_fcnet.pt", map_location=device))

<All keys matched successfully>

## FCNet vs SparseFCNet

### Clean images

In [8]:
from sklearn.metrics import accuracy_score

yp = fcnet(X).max(dim=1)[1].numpy()
syp = sparse_fcnet(X).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.888 
SparseFCNet: 0.888 


### Atak BIM

In [9]:
noise = bim(fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.063 
SparseFCNet: 0.331 


In [10]:
noise = bim(sparse_fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.454 
SparseFCNet: 0.082 


### Atak FGSM

In [11]:
noise = fgsm(fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.0 
SparseFCNet: 0.002 


In [12]:
noise = fgsm(sparse_fcnet, nn.CrossEntropyLoss(), X, y, alpha=0.3)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.005 
SparseFCNet: 0.001 


### Atak LLC

In [13]:
noise = llc(fcnet, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.513 
SparseFCNet: 0.702 


In [14]:
noise = llc(sparse_fcnet, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

yp = fcnet(X+noise).max(dim=1)[1].numpy()
syp = sparse_fcnet(X+noise).max(dim=1)[1].numpy()

print("FCNet: {} ".format(accuracy_score(y.numpy(), yp)))
print("SparseFCNet: {} ".format(accuracy_score(y.numpy(), syp)))

FCNet: 0.803 
SparseFCNet: 0.627 


## CNN vs SparseCNN vs CnnSparseFc

### CNN
- 2x convolutional layer
    - activation: ReLU
- 3x fully connected layer
    - activation: ReLU

### SparseCNN 
- 2x sparse convolutional layer - 10% of weight
    - KWinners with activation top 50%
- 3x sparse fully connected layer - 10% of weight
    - KWinners with activation top 50%

### CnnSparseFc 
- 2x convolutional layer
    - KWinners with activation top 10%
- 3x sparse fully connected layer - 10% of weight
    - KWinners with activation top 10%

In [15]:
cnn = CNN()
sparse_cnn = SparseCNN()
cnn_sparse_fc = CNNSparseFC()

dense_flat_relu = DenseFlatReLU()

cnn.load_state_dict(torch.load("models/cnn.pt", map_location=device))
sparse_cnn.load_state_dict(torch.load("models/sparse_cnn.pt", map_location=device))
cnn_sparse_fc.load_state_dict(torch.load("models/cnn_sparse_fc.pt", map_location=device))

dense_flat_relu.load_state_dict(torch.load("models/dense_flat_relu.pt", map_location=device))

<All keys matched successfully>

### Clean images

In [16]:
from sklearn.metrics import accuracy_score

cnn_pred = cnn(X).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X).max(dim=1)[1].numpy()

dense_frelu_pred = dense_flat_relu(X).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

print("DenseFlatReLU: {} ".format(accuracy_score(y.numpy(), dense_frelu_pred)))

CNN: 0.912 
SparseCNN: 0.901 
CnnSparseFc: 0.917 
DenseFlatReLU: 0.522 


### Atak BIM

In [17]:
noise = bim(cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.5 
SparseCNN: 0.755 
CnnSparseFc: 0.544 


In [18]:
noise = bim(sparse_cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.792 
SparseCNN: 0.585 
CnnSparseFc: 0.686 


In [19]:
noise = bim(cnn_sparse_fc, nn.CrossEntropyLoss(), X, y, alpha=0.3, max_iter=4, clip_eps=8/255, do_clip=True)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.831 
SparseCNN: 0.851 
CnnSparseFc: 0.442 


### Atak FGSM

In [20]:
noise = fgsm(cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.24 
SparseCNN: 0.452 
CnnSparseFc: 0.202 


In [21]:
noise = fgsm(sparse_cnn, nn.CrossEntropyLoss(), X, y, alpha=0.3)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.318 
SparseCNN: 0.366 
CnnSparseFc: 0.216 


In [22]:
noise = fgsm(cnn_sparse_fc, nn.CrossEntropyLoss(), X, y, alpha=0.3)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.306 
SparseCNN: 0.431 
CnnSparseFc: 0.174 


### Atak LLC

In [23]:
noise = llc(cnn, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.889 
SparseCNN: 0.885 
CnnSparseFc: 0.831 


In [24]:
noise = llc(sparse_cnn, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.9 
SparseCNN: 0.893 
CnnSparseFc: 0.792 


In [25]:
noise = llc(cnn_sparse_fc, nn.CrossEntropyLoss(), X, max_iter=4, alpha=0.3, clip_eps=8/255, do_clip=True)

cnn_pred = cnn(X+noise).max(dim=1)[1].numpy()
scnn_pred = sparse_cnn(X+noise).max(dim=1)[1].numpy()
cnn_s_fc_pred = cnn_sparse_fc(X+noise).max(dim=1)[1].numpy()

print("CNN: {} ".format(accuracy_score(y.numpy(), cnn_pred)))
print("SparseCNN: {} ".format(accuracy_score(y.numpy(), scnn_pred)))
print("CnnSparseFc: {} ".format(accuracy_score(y.numpy(), cnn_s_fc_pred)))

CNN: 0.889 
SparseCNN: 0.886 
CnnSparseFc: 0.725 
